# LangChain: Memory

## Outline
* ConversationBufferMemory
* ConversationBufferWindowMemory
* ConversationTokenBufferMemory
* ConversationSummaryMemory

In [1]:
## Load you env variables
import os
from dotenv import load_dotenv,find_dotenv
import openai
import anthropic
from anthropic import Anthropic

_ = load_dotenv(find_dotenv(filename='.env'))

## ConversationBufferMemory

In [3]:
from langchain_openai import ChatOpenAI
from langchain.chains import  ConversationChain
from langchain.memory import ConversationBufferMemory

In [4]:
llm_model = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.1,
    max_tokens=1024,
)
## Specify the buffer memory for storing the chat History
memory = ConversationBufferMemory()
## Start a conversation chains that take into consideration a buffer memory 
chatConversation = ConversationChain(
    llm=llm_model,
    memory=memory,
    verbose=True
)

In [5]:
## Start a conversation by asking a question to the LLM
chatConversation.predict(input="Hello There assistant , my name is mehdox how are you ?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hello There assistant , my name is mehdox how are you ?
AI:

> Finished chain.


"Hello Mehdox! I'm doing well, thank you for asking. How can I assist you today?"

In [6]:
chatConversation.predict(input="Can you guesse what is my name ?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hello There assistant , my name is mehdox how are you ?
AI: Hello Mehdox! I'm doing well, thank you for asking. How can I assist you today?
Human: Can you guesse what is my name ?
AI:

> Finished chain.


'Based on the information you provided, your name is Mehdox.'

In [9]:
## Conversation Chains Chat History 
memory.load_memory_variables({})

{'history': "Human: Hello There assistant , my name is mehdox how are you ?\nAI: Hello Mehdox! I'm doing well, thank you for asking. How can I assist you today?\nHuman: Can you guesse what is my name ?\nAI: Based on the information you provided, your name is Mehdox."}

## ConversationBufferWindowMemory

In [17]:
from langchain.memory import  ConversationBufferWindowMemory
## keeping a sliding window of the most recent interactions (k=Window size/input-output), so the buffer does not get too large.
bufferWindowMemory = ConversationBufferWindowMemory(k=1)

In [18]:
bufferWindowMemory.save_context(
    {
        "input":"Hello There ChatGPT how are you ?"
    },
    {
        "output": "fine thank you how can i assist you today"
    }
)
bufferWindowMemory.save_context(
    {
        "input": "Not much, just hanging"
    },
    {
        "output": "Cool"
    })

In [16]:
bufferWindowMemory.load_memory_variables({})

{'history': 'Human: Hello There ChatGPT how are you ?\nAI: fine thank you how can i assist you today\nHuman: Not much, just hanging\nAI: Cool'}

In [20]:
## Start a conversation chains that take into consideration a buffer Window memory 
chatBufferWindowConversation = ConversationChain(
    llm=llm_model,
    memory=bufferWindowMemory,
    verbose=True
)

In [21]:
chatBufferWindowConversation.predict(input="Hello there assistant , my name is mehdox remember me ok ?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Not much, just hanging
AI: Cool
Human: Hello there assistant , my name is mehdox remember me ok ?
AI:

> Finished chain.


'Hello Mehdox! Of course, I remember you. How can I assist you today?'

In [22]:
chatBufferWindowConversation.predict(input="give me the result of this math expression exp(0)")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hello there assistant , my name is mehdox remember me ok ?
AI: Hello Mehdox! Of course, I remember you. How can I assist you today?
Human: give me the result of this math expression exp(0)
AI:

> Finished chain.


'The result of the math expression exp(0) is 1. The exponential function raised to the power of 0 always equals 1. Is there anything else you would like to know?'

In [23]:
chatBufferWindowConversation.predict(input="I am mehdox , do you rememer me ?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: give me the result of this math expression exp(0)
AI: The result of the math expression exp(0) is 1. The exponential function raised to the power of 0 always equals 1. Is there anything else you would like to know?
Human: I am mehdox , do you rememer me ?
AI:

> Finished chain.


"Hello Mehdox! I'm sorry, but I don't have the ability to remember individual users. Is there anything else you would like to know or talk about?"

## ConversationTokenBufferMemory

In [70]:
## What if the buffer memory is controlled based on the number of tokens instead of windows size
from langchain.memory import ConversationTokenBufferMemory

tokenBufferMemory = ConversationTokenBufferMemory(
    llm=llm_model,
    max_token_limit=40,
)
chatTokenBufferMemory = ConversationChain(
    llm=llm_model,
    memory=tokenBufferMemory,
    verbose=True
)

In [71]:
tokenBufferMemory.save_context(
    {
        "input":"Langchain is what ?"
    },
    {
        "output":"Amazing Framework for LLM apps"
    }
)
tokenBufferMemory.save_context(
     {
        "input":"Azure data factory is what ?"
    },
    {
        "output":"is a ETL Cloud based service provided by azure cloud"
    }
)

In [72]:
tokenBufferMemory.load_memory_variables({})

{'history': 'AI: Amazing Framework for LLM apps\nHuman: Azure data factory is what ?\nAI: is a ETL Cloud based service provided by azure cloud'}

## ConversationSummaryMemory

In [95]:
from langchain.memory import ConversationSummaryBufferMemory
summaryMemory = ConversationSummaryBufferMemory(
    llm=llm_model,
    max_token_limit=100
)

In [96]:
# create a long string
schedule = "There is a meeting at 8am with your product team. \
You will need your powerpoint presentation prepared. \
9am-12pm have time to work on your LangChain \
project which will go quickly because Langchain is such a powerful tool. \
At Noon, lunch at the italian resturant with a customer who is driving \
from over an hour away to meet you to understand the latest in AI. \
Be sure to bring your laptop to show the latest LLM demo."

summaryMemory.save_context(
    {"input": "Hello"}, 
    {"output": "What's up"}
    )
summaryMemory.save_context(
    {"input": "Not much, just hanging"},
                    {"output": "Cool"})
summaryMemory.save_context({"input": "What is on the schedule today?"}, 
                    {"output": f"{schedule}"})

In [97]:
summaryMemory.load_memory_variables({})

{'history': "System: The human greets the AI and asks about the schedule for the day. The AI informs the human about a morning meeting with the product team, work on the LangChain project, and a lunch meeting with a customer interested in AI. The AI emphasizes the importance of being prepared for the day's activities."}

In [98]:
summaryMemoryChat = ConversationChain(
    llm=llm_model,
    memory=summaryMemory,
    verbose=True
)

In [99]:
summaryMemoryChat.predict(input="What is the main task in this noon ?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human greets the AI and asks about the schedule for the day. The AI informs the human about a morning meeting with the product team, work on the LangChain project, and a lunch meeting with a customer interested in AI. The AI emphasizes the importance of being prepared for the day's activities.
Human: What is the main task in this noon ?
AI:

> Finished chain.


"The main task for this noon is the lunch meeting with a customer interested in AI. It's a great opportunity to showcase our expertise and potentially secure a new partnership. I recommend preparing some key talking points and examples of our successful AI projects to impress the customer."

In [102]:
summaryMemory.load_memory_variables({})

{'history': "System: The human greets the AI and asks about the schedule for the day. The AI informs the human about a morning meeting with the product team, work on the LangChain project, and a lunch meeting with a customer interested in AI. The AI emphasizes the importance of being prepared for the day's activities.\nHuman: What is the main task in this noon ?\nAI: The main task for this noon is the lunch meeting with a customer interested in AI. It's a great opportunity to showcase our expertise and potentially secure a new partnership. I recommend preparing some key talking points and examples of our successful AI projects to impress the customer."}